In [ ]:
# Use the dotenv library to read environment variable overrides from the .env file.
# Then define and initialize variables from this config. 

import os
from dotenv import load_dotenv
from pysrc.services.config_service import ConfigService 

load_dotenv(override=True)
print('load_dotenv completed')

aoai_endpoint = ConfigService.azure_openai_url()
aoai_api_key = ConfigService.azure_openai_key()
aoai_completions_depl = ConfigService.azure_openai_completions_deployment()
aoai_embeddings_depl = ConfigService.azure_openai_embeddings_deployment()
mongo_vcore_conn_str = ConfigService.mongo_vcore_conn_str()
graph_source_db = ConfigService.graph_source_db()
documents_container = ConfigService.documents_container()
mem_container_name = 'mem1'

print('aoai_endpoint:  {}'.format(aoai_endpoint))
print('aoai_api_key:   {}'.format(aoai_api_key))
print('aoai_completions_depl: {}'.format(aoai_completions_depl))
print('aoai_embeddings_depl:  {}'.format(aoai_embeddings_depl))
print('mongo_vcore_conn_str:  {}'.format(mongo_vcore_conn_str))
print('graph_source_db:       {}'.format(graph_source_db))
print('documents_container:   {}'.format(documents_container))
print('mem_container_name:    {}'.format(mem_container_name))




In [ ]:
# Confirm local file IO with class FS

from pysrc.util.fs import FS

infile = 'requirements.in'
lines = FS.read_lines(infile)
print('{} lines read from file {}'.format(len(lines), infile))

In [ ]:
# Create Helper Functions

import json
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.memory_store_base import MemoryStoreBase

async def upsert_data_to_memory_store(
    memory: SemanticTextMemory, store: MemoryStoreBase, data_file_path: str
) -> None:
    """
    This asynchronous function takes two memory stores and a data file path as arguments.
    It is designed to upsert (update or insert) data into the memory stores from the data file.

    Args:
        kernel_memory_store (callable): A callable object that represents the kernel memory store where data will be upserted.
        memory_store (callable): A callable object that represents the memory store where data will be upserted.
        data_file_path (str): The path to the data file that contains the data to be upserted.

    Returns:
        None. The function performs an operation that modifies the memory stores in-place.
    """
    with open(file=data_file_path, mode="r", encoding="utf-8") as f:
        data = json.load(f)
        n = 0
        for item in data:
            n += 1
            # check if the item already exists in the memory store
            # if the id doesn't exist, it throws an exception
            try:
                already_created = bool(
                    await store.get(
                        mem_container_name, item["id"], with_embedding=True
                    )
                )
            except Exception:
                already_created = False

            # if the record doesn't exist, we generate embeddings and save it to the database
            if not already_created:
                await memory.save_information(
                    collection=mem_container_name,
                    id=item["id"],
                    # the embedding is generated from the text field
                    text=item["content"],
                    description=item["title"],
                )
                print(
                    "Generating embeddings and saving new item:",
                    n,
                    "/",
                    len(data),
                    end="\r",
                )
            else:
                print("Skipping item already exits:", n, "/", len(data), end="\r")
    

In [ ]:
# Setup Semantic Kernel and the chat_completion and text_embedding services
# within the kernel

import semantic_kernel as sk

from pysrc.services.config_service import ConfigService 

# Intialize the kernel

kernel = sk.Kernel()

from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextEmbedding,
)

# Adding chat_completion service (AzureChatCompletion)
kernel.add_service(
    AzureChatCompletion(
        service_id="chat_completion",
        deployment_name=aoai_completions_depl,
        endpoint=aoai_endpoint,
        api_key=aoai_api_key
    )
)
print("Added Azure OpenAI Chat Service...")

# Adding text_embedding service (AzureTextEmbedding)
kernel.add_service(
    AzureTextEmbedding(
        service_id="text_embedding",
        deployment_name=aoai_embeddings_depl,
        endpoint=aoai_endpoint,
        api_key=aoai_api_key
    )
)
print("Added Azure OpenAI Embedding Generation Service...")

In [ ]:
# Configure AzureCosmosDBMemoryStore

collection_name = documents_container

# Vector search index parameters
index_name = "vectorSearchIndex"
vector_dimensions = (
    1536  # text-embedding-ada-002 uses a 1536-dimensional embedding vector
)
num_lists = 20
similarity = "COS"  # cosine distance
cosmos_api = "mongo-vcore"

from semantic_kernel.connectors.memory.azure_cosmosdb import (
    AzureCosmosDBMemoryStore,
)

# aoai_endpoint = ConfigService.azure_openai_url()
# aoai_api_key = ConfigService.azure_openai_key()
# aoai_completions_depl = ConfigService.azure_openai_completions_deployment()
# aoai_embeddings_depl = ConfigService.azure_openai_embeddings_deployment()
# mongo_vcore_conn_str = ConfigService.mongo_vcore_conn_str()
# graph_source_db = ConfigService.graph_source_db()
# documents_container = ConfigService.documents_container()
# mem_container_name = 'mem1'

# db.runCommand({
#   createIndexes: 'mem1',
#   indexes: [
#     {
#       name: 'vectorSearchIndex',
#       key: {
#         "embeddings": "cosmosSearch"
#       },
#       cosmosSearchOptions: {
#         kind: 'vector-ivf',
#         numLists: 20,
#         similarity: 'COS',
#         dimensions: 1536
#       }
#     }
#   ]
# });

print("Creating or updating Azure Cosmos DB Memory Store...")
# create azure cosmos db for mongo db vcore api store and collection with vector ivf
# currently, semantic kernel only supports the ivf vector kind
store = await AzureCosmosDBMemoryStore.create(
    cosmos_connstr=mongo_vcore_conn_str,
    cosmos_api=cosmos_api,
    database_name=graph_source_db,
    collection_name=mem_container_name,
    index_name=index_name,
    vector_dimensions=vector_dimensions,
    num_lists=num_lists,
    similarity=similarity
)
print("Finished updating Azure Cosmos DB Memory Store...")

In [ ]:
# Add the created memory store to the semantic kernel instance.

from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin

memory = SemanticTextMemory(storage=store, embeddings_generator=kernel.get_service("text_embedding"))
kernel.import_plugin_from_object(TextMemoryPlugin(memory), "TextMemoryPluginACDB")
print("Registered Azure Cosmos DB Memory Store...")


In [ ]:
# Add data to the memory store

print("Upserting data to Azure Cosmos DB Memory Store...")
await upsert_data_to_memory_store(memory, store, "./data/text-sample.json")


In [ ]:
# Test the Vector DB

# each time it calls the embedding model to generate embeddings from your query
query_term = "What is Azure Database for Managed Instances?"
result = await memory.search(mem_container_name, query_term)

print(
    f"Result is: {result[0].text}\nRelevance Score: {result[0].relevance}\nFull Record: {result[0].additional_metadata}"
)


In [ ]:
# Create a string prompt

prompt = """
    You are a chatbot that can have a conversations about any topic related to the provided context.
    Give explicit answers from the provided context or say 'I don't know' if it does not have an answer.
    provided context: {{$db_record}}

    User: {{$query_term}}
    Chatbot:"""


In [ ]:
# Create the OpenAITextPromptExecutionSettings

import semantic_kernel.connectors.ai.open_ai as sk_oai

execution_settings = sk_oai.OpenAITextPromptExecutionSettings(
   service_id="chat_completion",
    ai_model_id=aoai_completions_depl,
    max_tokens=500,
    temperature=0.0,
    top_p=0.5
)

print('OpenAITextPromptExecutionSettings created')


In [ ]:
# Create the PromptTemplateConfig

from semantic_kernel.prompt_template.input_variable import InputVariable

chat_prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="grounded_response",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="db_record", description="The database record", is_required=True),
        InputVariable(name="query_term", description="The user input", is_required=True),
    ],
    execution_settings=execution_settings
)
print('PromptTemplateConfig created')

In [ ]:
# Create a chat function

chat_function = kernel.create_function_from_prompt(
    prompt=prompt,
 function_name= "ChatGPTFunc2", plugin_name="chatGPTPlugin2", prompt_template_config=chat_prompt_template_config
)
print('chat function created')

In [ ]:

completions_result = await kernel.invoke(chat_function, sk.KernelArguments(query_term=query_term, db_record=result[0].additional_metadata))

print(completions_result)

